Case Studies

Project: 1

Group: 3

Group Members:
 - Muhammad Raafey Tariq (231806)
 - Farrukh Ahmed (230614)
 - Amirreza Khamehchin Khiabani (230891)
 - Aymane Hachcham (236392)


Requirements:
 - numpy==1.24.2
 - matplotlib==3.7.1
 - seaborn==0.12.2
 - pandas==2.0.0
 - openpyxl==3.1.2

Installation Commands (One-time only)
 - pip install pandas==2.0.0
 - pip install numpy==1.24.2
 - pip install seaborn==0.12.2
 - pip install matplotlib==3.7.1
 - pip install openpyxl==3.1.2

Imports and Libraries

In [341]:
import pandas as pd
import numpy as np
import itertools
import pprint
import random

# used for the graphs
import seaborn as sns

import os
sns.set(font_scale = 1.2)

# used for plotting
from matplotlib import pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor


import matplotlib

# setting font to 'Times New Roman'
matplotlib.rcParams["font.family"] = "Times New Roman"
matplotlib.rcParams.update({'font.size': 16})
%matplotlib inline

Global Variables and Constants

Importing Data

In [342]:
file_path = "../styrian_health_data.xlsx"
sheet_name = "Sheet 1"
data_df = pd.read_excel(file_path, sheet_name=sheet_name)

c:\Users\Raafe\AppData\Local\Programs\Python\Python38\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Reformating Columns to Correct Data Types and dropping nans

In [343]:
# function to format variable types, remove nans, shuffle data
def format_variables(data, to_filter=[]):
    data_df = data.copy()
    data_df.postleitzahl = data_df.postleitzahl.astype('str')
    data_df.geburtsjahr = data_df.geburtsjahr.astype('Int64')
    # data_df.befinden = data_df.befinden.astype('Int64')
    data_df.messwert_bp_sys = pd.to_numeric(data_df.messwert_bp_sys)
    data_df.messwert_bp_dia = pd.to_numeric(data_df.messwert_bp_dia)
    data_df.schaetzwert_bp_sys = pd.to_numeric(data_df.schaetzwert_bp_sys)
    data_df.schaetzwert_by_dia = pd.to_numeric(data_df.schaetzwert_by_dia)

    # adding variable for is_local
    mask = data_df.gemeinde.isna() & data_df.bezirk.isna() & data_df.bundesland.isna()
    data_df["is_local_resident"] = True
    data_df.loc[mask, "is_local_resident"] = False

    # adding variable for age
    age =  data_df["zeit"].dt.year - pd.to_datetime(data_df['geburtsjahr'], format='%Y').dt.year
    data_df["age"] = age.astype("Int64")

    # adding variable for age group
    data_df["age_group"] = pd.cut(data_df.age, bins=[0,12,19,65,130],labels=['children', 'teenager', 'adult','65 over'])
    data_df["age_group"] = data_df.age_group.astype(str)

    #replacing nans for variables

    data_df.loc[data_df.geschlecht.isna() == True, 'raucher'] = "unknown"
    data_df.loc[data_df.geschlecht.isna() == True, 'blutzucker_bekannt'] = "unknown"
    data_df.loc[data_df.geschlecht.isna() == True, 'cholesterin_bekannt'] = "unknown"
    data_df.loc[data_df.geschlecht.isna() == True, 'in_behandlung'] = "unknown"
    data_df.loc[data_df.geschlecht.isna() == True, 'befinden'] = "unknown"
    data_df.loc[data_df.age_group == "nan", 'age_group'] = "unknown"

    data_df.loc[mask, 'gemeinde'] = "not applicable"
    data_df.loc[mask, 'bezirk'] = "not applicable"
    data_df.loc[mask, 'bundesland'] = "not applicable"
    data_df.loc[mask, 'postleitzahl'] = "not applicable"
    data_df.loc[data_df.postleitzahl == "nan", 'postleitzahl'] = "unknown"

    # creating variables for missing values in bp

    data_df["is_missing_schaetzwert_bp_sys"] = False
    data_df.loc[data_df.schaetzwert_bp_sys.isna() == True, "is_missing_schaetzwert_bp_sys"] = True
    data_df["is_missing_schaetzwert_by_dia"] = False
    data_df.loc[data_df.schaetzwert_by_dia.isna() == True, "is_missing_schaetzwert_by_dia"] = True

    # removing useless variables
    data_df.drop(data_df[data_df.age > 100].index, inplace=True)
    data_df.drop(data_df[data_df.age < 15].index, inplace=True)

    data_df.loc[data_df.geschlecht.isna() == True, 'geschlecht'] = "unknown"

    data_df = data_df.dropna()

    if len(to_filter) > 0:
        data_df = data_df.drop(to_filter, axis=1)

    data_df.befinden = data_df.befinden.astype('Int64')

    data_df['befinden'] = data_df['befinden'].astype(object)
    data_df['messwert_bp_sys'] = data_df['messwert_bp_sys'].astype(float)
    data_df['messwert_bp_dia'] = data_df['messwert_bp_dia'].astype(float)
    data_df['geschlecht'] = data_df['geschlecht'].astype(object)
    data_df['is_local_resident'] = data_df['is_local_resident'].astype(object)

    # shuffling data with fixed seed
    data_df = data_df.sample(frac=1, random_state=1).reset_index(drop=True)

    # separating var types
    cat_feat_list = []
    num_feat_list = []

    for var in data_df.columns:
        if data_df[var].dtype == object:
            cat_feat_list.append(var)
        else:
            num_feat_list.append(var)

    return data_df, cat_feat_list, num_feat_list


# function that converts cat columns in df to one-hot encoding
def encode_data(df, cat_feat_list, num_feat_list):
    one_hot_data = pd.get_dummies(df[cat_feat_list], drop_first=True, dtype=int)

    for var in num_feat_list:
        one_hot_data[var] = df[var] 
    
    return one_hot_data

# function to separate target from dataframe
def separate_target(data, target):
    df = data.copy()
    Y = df[target]
    del df[target]
    X = df

    return X, Y

def adjusted_r2(r_2, n, k):
    return 1 - (1-r_2)*(n-1)/(n-k-1)


def compute_metrics(pred, target, num_feats):
    r_2 = r2_score(pred, target)
    mse = mean_squared_error(pred, target)
    adj_r2 = adjusted_r2(r_2, len(pred), num_feats)
    return {
        "r_2": r_2,
        "adjusted_r_2": adj_r2,
        "mse": mse
    }

def fit_and_eval_regression_tree(X_train, Y_train, X_test, params):
    model = DecisionTreeRegressor(criterion=params["criterion"])
    model.fit(X_train, Y_train)
    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)

    return train_predictions, test_predictions, model


def fit_model(X_train, Y_train, X_test, Y_test, model_type, params):
    num_feats = len(X_train.columns)
    train_results = None
    test_results = None
    model = None

    if model_type == "DecisionTreeRegressor":
        train_predictions, test_predictions, model = fit_and_eval_regression_tree(X_train, Y_train, X_test, params)
        train_results = compute_metrics(train_predictions, Y_train, num_feats)
        test_results = compute_metrics(test_predictions, Y_test, num_feats)

    return train_results, test_results, model


def best_subset_selection(features, criterion, X_train, Y_train, X_test, Y_test, model_type, params):
    if criterion == "mse":
        best_val = np.inf
    elif criterion in ["adjusted_r_2", "r_2"]:
        best_val = -np.inf

    best_train_results = None
    best_model = None
    best_test_results = None
    best_features = None
    n_features = len(features)


    for i in range(1, n_features):
        print("\nNum features: ", i, "=======================================================")

        for j in range(n_features):
            current_features = features[j:j+i]
            if len(current_features) < i:
                break

            X_train_curr = X_train[current_features]
            X_test_curr = X_test[current_features]
            
            train_results, test_results, model = fit_model(X_train_curr, Y_train, X_test_curr, Y_test, model_type, params)

            print("\nFeatures: ", current_features)
            print("Train Results: ", train_results)
            print("Test Results: ", test_results)

            condition = False
            if criterion == "mse":
                condition = test_results[criterion] < best_val
            elif criterion in ["adjusted_r_2", "r_2"]:
                condition = test_results[criterion] > best_val   

            if condition:
                best_val = test_results[criterion]
                best_model = model
                best_features = current_features
                best_train_results = train_results
                best_test_results = test_results
    
    print("\nBest Model: ")
    print("Features: ", best_features)
    print("Train Results: ", best_train_results)
    print("Test Results: ", best_test_results)
    
    return best_model, best_train_results, best_test_results


In [344]:
# variables that are dropped 
to_filter = ["id", "zeit", "postleitzahl", "gemeinde", "bezirk", "geburtsjahr",
              "is_missing_schaetzwert_bp_sys", "is_missing_schaetzwert_by_dia", "terminal"]
data_df, cat_feat_list, num_feat_list = format_variables(data_df, to_filter=to_filter)

# one hot encoding cat variables to prep data for Decision Tree
# ordinal variables and nominal are treated the same in trees, but need to be careful in Lin models

encoded_data_df = encode_data(data_df, cat_feat_list, num_feat_list)
encoded_train_set, encoded_test_set = train_test_split(encoded_data_df, test_size=0.3)

In [345]:
print("Size of training data: ", len(encoded_train_set))
print("Size of testing data: ", len(encoded_test_set))
print("Features used: ", data_df.columns)

Size of training data:  10381
Size of testing data:  4450
Features used:  Index(['bundesland', 'befinden', 'geschlecht', 'raucher', 'blutzucker_bekannt',
       'cholesterin_bekannt', 'in_behandlung', 'schaetzwert_bp_sys',
       'schaetzwert_by_dia', 'messwert_bp_sys', 'messwert_bp_dia',
       'is_local_resident', 'age', 'age_group'],
      dtype='object')


https://scikit-learn.org/stable/modules/tree.html#tree

 - scikit-learn uses an optimized version of the CART algorithm, does not support categorical variables
 - BIC cannot be computed as it depends on likelihood, cannot compute that for RegressionTree as it does not assume a conditional dist of data

In [346]:
target = "messwert_bp_sys"
X_train, Y_train = separate_target(encoded_train_set, target)
X_test, Y_test = separate_target(encoded_test_set, target)

In [347]:
model_type = "DecisionTreeRegressor"
model_params = {
    "criterion": "squared_error"
}
criterion = "r_2"
features = list(X_train.columns)
train_results, test_results, model = best_subset_selection(features, criterion, X_train, Y_train, X_test, Y_test,
                                                     model_type, model_params)


Num features:  1 =======================================================

Features:  ['bundesland_Kärnten']
Train Results:  {'r_2': -273965.58811910864, 'adjusted_r_2': -273991.9843603765, 'mse': 373.84291209233976}
Test Results:  {'r_2': -238925.53623493004, 'adjusted_r_2': -238979.2517331843, 'mse': 361.3996312381961}

Features:  ['bundesland_Niederösterreich']
Train Results:  {'r_2': -8889.88012179348, 'adjusted_r_2': -8890.736743830457, 'mse': 373.8022333159601}
Test Results:  {'r_2': -7484.126683008325, 'adjusted_r_2': -7485.809490266196, 'mse': 361.57710521449144}

Features:  ['bundesland_Oberösterreich']
Train Results:  {'r_2': -5958.054184366736, 'adjusted_r_2': -5958.62832967788, 'mse': 373.7815516694061}
Test Results:  {'r_2': -4647.735644466073, 'adjusted_r_2': -4648.780773882545, 'mse': 361.27924140142915}

Features:  ['bundesland_Salzburg']
Train Results:  {'r_2': -3700.2952975169233, 'adjusted_r_2': -3700.6519113812183, 'mse': 373.7433002959859}
Test Results:  {'r_2': -5